In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jeyasriramg","key":"eb3ac18aadc36f62ea203bb3027ca2c1"}'}

In [7]:
! cp kaggle.json ~/.kaggle/

In [8]:
! chmod 600 ~/.kaggle/kaggle.json

In [11]:
!kaggle datasets download -d shashwatwork/identifying-disease-in-tea-leafs 

100% 740M/740M [00:21<00:00, 32.0MB/s]
100% 740M/740M [00:21<00:00, 36.6MB/s]


In [12]:
!unzip /content/identifying-disease-in-tea-leafs.zip

Archive:  /content/identifying-disease-in-tea-leafs.zip
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143242.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143328.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143336.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143344.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143352.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143401.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143424.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143433.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143441.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143451.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143501.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143511.jpg  
  inflating: tea sickness dataset/Anthracnose/IMG_20220503_143525.jpg  
  inflat

In [13]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


**Adding prepocessing layers to the font of vgg**

In [14]:
imageSize=[224,224];
vgg = VGG16(input_shape=imageSize + [3], weights='imagenet',include_top=False)

58889256/58889256 [==============================] - 2s 0us/step


In [15]:
#don't train existinng weights
for layer in vgg.layers:
  layer.trainable = False

In [16]:
#our layers you can add more if you want
x = Flatten()(vgg.output)

**adding layers**

In [17]:
#our layers you can add mor if you want
#adding flatten layers
x = Flatten()(vgg.output)

In [18]:
#adding output layer
prediction = Dense(8, activation='softmax')(x)

**define model**

In [20]:
#create model
model = Model(inputs=vgg.input, outputs=prediction)

In [21]:
#view the structure of the model
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

**configuration**

In [22]:
#tell the modl what cost and optimitation method to use
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'], run_eagerly=True)

**import image datagenerator linrary**

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [24]:

train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [27]:
trainPath="/content/tea sickness dataset"
testPath="/content/tea sickness dataset"
training_set = train_datagen.flow_from_directory(trainPath,
                                                target_size = (224, 224),
                                                batch_size = 32,
                                                class_mode = 'categorical'
                                                )
test_set = test_datagen.flow_from_directory(testPath,
                                                target_size = (224, 224),
                                                batch_size = 1,
                                                class_mode = 'categorical'
                                                )

Found 885 images belonging to 8 classes.
Found 885 images belonging to 8 classes.


In [28]:
training_set.class_indices

{'Anthracnose': 0,
 'algal leaf': 1,
 'bird eye spot': 2,
 'brown blight': 3,
 'gray light': 4,
 'healthy': 5,
 'red leaf spot': 6,
 'white spot': 7}

**train the model**

In [29]:
history = model.fit(x=training_set,validation_data=test_set,epochs =5)

Epoch 1/5
28/28 [==============================] - 200s 6s/step - loss: 1.7358 - accuracy: 0.4486 - val_loss: 0.9870 - val_accuracy: 0.6294
Epoch 2/5
28/28 [==============================] - 171s 6s/step - loss: 0.9199 - accuracy: 0.6531 - val_loss: 0.6057 - val_accuracy: 0.7887
Epoch 3/5
28/28 [==============================] - 166s 6s/step - loss: 0.6308 - accuracy: 0.7774 - val_loss: 0.4391 - val_accuracy: 0.8486
Epoch 4/5
28/28 [==============================] - 169s 6s/step - loss: 0.4810 - accuracy: 0.8339 - val_loss: 0.3495 - val_accuracy: 0.8904
Epoch 5/5
28/28 [==============================] - 168s 6s/step - loss: 0.4025 - accuracy: 0.8667 - val_loss: 0.2919 - val_accuracy: 0.9254


In [30]:
model.save('vgg-16-Tea-leaves-disease-modl.h5')

In [31]:
#import load_model class for loading h5 file
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np

**save the model**

In [32]:
%ls


 identifying-disease-in-tea-leafs.zip  'tea sickness dataset'/
 kaggle.json                            vgg-16-Tea-leaves-disease-modl.h5
 sample_data/


In [33]:
model=load_model('vgg-16-Tea-leaves-disease-modl.h5')

In [38]:
img=image.load_img('/content/tea sickness dataset/healthy/UNADJUSTEDNONRAW_thumb_24b.jpg',target_size= (224,224))
 #convert image to array form at
x=image.img_to_array(img) 
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
output=np.argmax(model.predict(img_data), axis=1)
index= ['Anthra cnose ',
'algal leaf',
'bird eye spot ',
'brown blight ',
'gray light',
'healthy ',
'red leaf spot',
'white spot']
result = str(index [output [0]])
result

1/1 [==============================] - 0s 23ms/step


'healthy '